# 📐별의 크기 예측 모형 모델링(분류과제) - simple version

- 빅데이터 분석기사 실기 연습을 위해 제가 만든 자료입니다.
- 데이터셋은 kaggle의 데이터셋을 제가 다시 가공한 뒤 X_train, y_train, X_test로 분리하였습니다. 원본 데이터셋은 다음의 주소에서 확인하실 수 있습니다 : https://www.kaggle.com/datasets/vinesmsuic/star-categorization-giants-and-dwarfs
- 이 과제는 여러가지 변수들을 사용해서 별의 크기를 0 또는 1로 예측합니다.
- y_train 데이터셋은 별의 크기가 Giant인 경우 Targetclass는 1로 표기되어 있으며, 별의 크기가 Dwarf인 경우 Targetclass는 0으로 표기되어 있습니다.

### 📐 문제
- X_train.csv를 사용하여 별의 크기(Target class)를 예측하는 모형을 만들고, x_test.csv에 적용하여 별의 크기를 예측값을 0 또는 1로 만들어 .csv 파일로 제출하시오.(평가 지표는 f1 score이다)

### 📐 힌트
- 예측값을 0 또는 1로 분류하기 위해서는 predict_proba가 아닌 predict를 사용합니다.

# 1. 필요한 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np

# 2. 데이터 불러오기

In [2]:
x_train = pd.read_csv('./star_X_train.csv')
x_test = pd.read_csv('./star_X_test.csv')
y_train = pd.read_csv('./star_y_train.csv')

In [3]:
x_train.head(1)

,Vmag,Plx,e_Plx,B-V,Amag
0,6.86,1.79,0.81,0.047,13.124266


In [4]:
y_train.head(1)

,TargetClass
0,0


# 3. info() 함수로 기본적인 정보 확인
- 행의 수, 열의 수, 데이터의 결측치, 데이터 유형을 알아봅니다.

In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    2547 non-null   float64
 1   Plx     2549 non-null   float64
 2   e_Plx   2549 non-null   float64
 3   B-V     2548 non-null   float64
 4   Amag    2549 non-null   float64
dtypes: float64(5)
memory usage: 99.7 KB


In [6]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   TargetClass  2549 non-null   int64
dtypes: int64(1)
memory usage: 20.0 KB


In [7]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    1093 non-null   float64
 1   Plx     1093 non-null   float64
 2   e_Plx   1093 non-null   float64
 3   B-V     1093 non-null   float64
 4   Amag    1093 non-null   float64
dtypes: float64(5)
memory usage: 42.8 KB


- x_train 데이터셋에 결측치가 존재합니다. Vmag 컬럼에 2개의 결측치, B-V 컬럼에 1개의 결측치가 존재합니다.

# 4. 결측치 처리
- x_train의 두 개의 컬럼에 존재하는 결측치들을 처리합니다.
- 가장 간단한 방법으로는 dropna()를 사용하여 결측치가 포함된 전체 행들을 제거하는 방법이 있습니다.
- 여기에서는 결측치를 평균값으로 대체하는 방법을 사용합니다. 
- 💦 평균값 계산에는 결측치가 포함된 행은 제외되므로 결측치로 인한 왜곡을 피할 수 있습니다.

In [8]:
x_Vmag_mean = x_train['Vmag'].mean()
x_BV_mean = x_train['B-V'].mean()
print(x_Vmag_mean)
print(x_BV_mean)

8.13064389477817
0.8296636577708006


In [9]:
x_train['Vmag'] = x_train['Vmag'].fillna(x_Vmag_mean)
x_train['B-V'] = x_train['B-V'].fillna(x_BV_mean)

In [10]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    2549 non-null   float64
 1   Plx     2549 non-null   float64
 2   e_Plx   2549 non-null   float64
 3   B-V     2549 non-null   float64
 4   Amag    2549 non-null   float64
dtypes: float64(5)
memory usage: 99.7 KB


- 결측치를 평균값으로 채운 후 더 이상 결측치는 보이지 않습니다.

# 5. describe() 함수로 요약 통계량 확인
- 이상치를 확인합니다.
- 💦 Plx 컬럼과 e_Plx 컬럼에 max값이 매우 큰 값이 존재하는 것을 확인할 수 있습니다.
- 이상치는 중앙값 또는 평균값으로 대체합니다.

In [11]:
x_train.describe()

,Vmag,Plx,e_Plx,B-V,Amag
count,2549.000000,2549.000000,2549.000000,2549.000000,2549.000000
mean,8.130644,7.533107,1.160416,0.829664,16.409761
std,1.316081,11.945021,0.738598,0.439688,2.390487
min,2.010000,-4.780000,0.470000,-0.165000,3.045150
25%,7.380000,2.680000,0.850000,0.488000,15.121679
50%,8.390000,4.840000,1.040000,0.874000,16.349780
75%,9.060000,8.620000,1.270000,1.137000,17.989560
max,12.380000,280.270000,22.220000,2.530000,29.249165


- Plx 컬럼의 max값을 보아 이상치로 판단됩니다. 이러한 값들이 얼마나 있는지 nlargest() 함수를 통해 컬럼에서 가장 큰 값 5개만 추출해봅니다.
- e_Plx 컬럼에도 max값을 보아 이상치로 판단됩니다. 이러한 값들이 얼마나 있는지 nlargest() 함수를 통해 컬럼에서 가장 큰 값 5개만 추출해봅니다.

In [12]:
print(x_train['Plx'].nlargest(5))
print(x_train['e_Plx'].nlargest(5))

2321    280.27
25      269.05
559     134.04
106     122.75
258     100.24
Name: Plx, dtype: float64
1025    22.22
67      10.40
1971     9.62
25       7.57
1427     7.28
Name: e_Plx, dtype: float64


- 💦 Plx 컬럼의 경우에 280.27과 269.05가 다른 값들에 비해 매우 크고, e_Plx 컬럼의 경우에 22.22가 다른 값들에 비해 매우 큰 것을 확인할 수 있습니다.
- 여기에선 이러한 값들만 이상치로 간주하고 다른 값으로 대체하겠습니다. 이상치는 일반적으로 IQR의 1.5배를 초과하거나 미만인 값을 이상치라고 하지만 간단한 계산을 위해 Plx 컬럼에선 위의 2개의 값, e_Plx 컬럼에선 위의 1개의 값만 이상치로 처리하겠습니다.
- 그리고 이상치들은 평균값으로 대체하는 방법을 사용하겠습니다.

In [13]:
Plx_mean = x_train['Plx'].mean()
e_Plx_mean = x_train['e_Plx'].mean()
x_train['Plx'] = np.where(x_train['Plx']>250, Plx_mean, x_train['Plx'])
x_train['e_Plx'] = np.where(x_train['e_Plx']>20, e_Plx_mean, x_train['e_Plx'])

In [14]:
print(x_train['Plx'].describe())

count    2549.000000
mean        7.323514
std         9.306173
min        -4.780000
25%         2.680000
50%         4.840000
75%         8.600000
max       134.040000
Name: Plx, dtype: float64


In [15]:
print(x_train['e_Plx'].describe())

count    2549.000000
mean        1.152154
std         0.609424
min         0.470000
25%         0.850000
50%         1.040000
75%         1.270000
max        10.400000
Name: e_Plx, dtype: float64


- 훨씬 감소된 max값을 확인할 수 있습니다. 

# 6. 학습용 데이터셋과 검증용 데이터셋으로 분리
- 분류과제인 경우에는 stratify = y_train['TargetClass']를 꼭 사용하는 것이 좋습니다.

In [16]:
from sklearn.model_selection import train_test_split

x_tr, x_valid, y_tr, y_valid = train_test_split(x_train, y_train['TargetClass'], 
                                                test_size = 0.3, stratify = y_train['TargetClass'])

# 7. 모델 학습

In [17]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()
modelRF.fit(x_tr, y_tr)

RandomForestClassifier()

# 8. 모델 평가
- 0 또는 1이라는 값으로 예측 결과를 도출해야 합니다.
- 💦 predict를 사용하여 계산합니다.

In [18]:
y_validation_pred = modelRF.predict(x_valid)

In [19]:
from sklearn.metrics import accuracy_score, f1_score

f1 = f1_score(y_valid, y_validation_pred)
accuracy = accuracy_score(y_valid, y_validation_pred)

print(f1)
print(accuracy)

0.883048620236531
0.8836601307189542


In [20]:
pred = modelRF.predict(x_test)

# 9. 파일 제출

In [21]:
pd.DataFrame({'TargetClass' : pred}).to_csv('star_submission.csv', index =  False)

In [22]:
df =  pd.read_csv('./star_submission.csv')
df

,TargetClass
0,0
1,0
2,1
3,0
4,0
...,...
1088,1
1089,0
1090,1
1091,0
